In [1]:
import networkx as nx
import pandas as pd
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import csv

In [2]:
string_addr = "../data/9606.protein.links.v11.5.noscores.threshold400.txt"
with open(string_addr) as network_f:
    string_network_edges = [x.strip().split(',') for x in network_f.readlines()]

# STRING edgelist to network
G = nx.Graph(string_network_edges)

In [3]:
data_table = pd.read_csv("../data/DepMap_CRISPR_proc.csv")
data_table.set_index('ensembl_peptide_id', inplace=True)
data_table.drop(columns=['Unnamed: 0'], inplace=True)
data_table.drop(columns=['hgnc_symbol'], inplace=True)
data_table.head()

,ACH-000159,ACH-000246,ACH-000250,ACH-000262,ACH-000272,ACH-000313,ACH-000411,ACH-000433,ACH-000459,ACH-000600,ACH-000649,ACH-000684,ACH-000709,ACH-000907,ACH-001687,ACH-002189
ensembl_peptide_id,,,,,,,,,,,,,,,,
ENSP00000263100,0.053875,0.047606,0.059083,0.121036,0.032755,0.064195,0.051242,0.093111,0.015883,0.018940,0.104524,0.017930,0.070982,0.049880,0.124364,0.052712
ENSP00000378868,0.075826,0.022988,0.024956,0.009310,0.047183,0.058432,0.008625,0.019556,0.022441,0.025182,0.021885,0.025388,0.023770,0.024116,0.045705,0.316216
ENSP00000323929,0.013374,0.012579,0.011173,0.009070,0.017651,0.015902,0.010458,0.007736,0.028978,0.011729,0.019071,0.022923,0.007634,0.019421,0.002200,0.035317
ENSP00000299698,0.023290,0.007506,0.007579,0.009041,0.010344,0.016108,0.008501,0.008902,0.022263,0.015625,0.003625,0.008477,0.018610,0.010819,0.000964,0.019762
ENSP00000475261,0.025995,0.046996,0.082759,0.067371,0.117483,0.102422,0.086093,0.069194,0.163348,0.092527,0.053591,0.016644,0.198497,0.048408,0.060264,0.232389


In [4]:
nodes_to_remove = [node for node in G.nodes() if node not in data_table.index]
print(len(nodes_to_remove))
G.remove_nodes_from(nodes_to_remove)

1423


In [5]:
# Convert to nested dictionary
df = data_table
data_table = data_table.to_dict(orient='index')

In [6]:
# Compute the average gene dependency scores for neighbors of a node
def average_dependency_scores(node, data_table):
    neighbors = list(G.neighbors(node))
    scores_sum = [0] * 16  # 16 cell lines
    valid_neighbors_count = 0
    
    for neighbor in neighbors:
        if neighbor in data_table:
            valid_neighbors_count += 1
            for idx, score in enumerate(data_table[neighbor].values()):
                scores_sum[idx] += score
                
    return [x / valid_neighbors_count if valid_neighbors_count != 0 else 0 for x in scores_sum]


average_scores = {}
for node in G.nodes():
    average_scores[node] = average_dependency_scores(node, data_table)


results_df = pd.DataFrame.from_dict(average_scores, orient='index', columns=df.columns)
results_df.to_csv("outputs/DepMap_PPI_neighbourhood_avg_scores.csv")